In [208]:
os.chdir('/Users/aaron/Documents/Uni/PhD/Paper_Ours/CD_MLs/Code/')
sys.path.insert(0,"/Users/aaron/Documents/Uni/PhD/Paper_Ours/CD_MLs/Code/script_onelist_colab.py")

In [209]:

import os
import keras
import json
import glob
import sys
import pandas as pd
import tensorflow as tf
from keras.layers import Input
import numpy as np
import argparse
#from keras_applications.resnext import ResNeXt50
from keras.utils.data_utils import get_file
#import face_recognition
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add,Normalization
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
! pip install keras_tuner
import keras_tuner as kt
from keras import backend as K
#sys.path.append('/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML')
import fnmatch
import re
import copy
import matplotlib.pyplot as plt
import scipy.signal
! pip install mpu
import mpu
from script_onelist_colab import norm_int,repair,csv_exp,sel_exp,maxima,import_raw,read_processed,read_processed_sel,correct,onelist,interpolate_mat,interpolation_spectra

def touch(fname):
    if os.path.exists(fname):
        os.utime(fname, None)
    else:
        os.mkdir(fname)

#folder='/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML/csv/'
#if not os.path.abspath('')=='/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML/csv':
#  os.chdir(folder)
#print(os.path.abspath(''))  
#touch(folder+'png')





In [210]:
datadict={}

In [211]:
def noise(source):
  noisy=copy.deepcopy(source)
  smax=np.amax(noisy[:,1:])
  for j in range(len(noisy[0,:])):
    for i in range(len(noisy[:,0])):
      if j>0:
        noisy[i,j]+=np.random.uniform(-2,2)
        noisy[i,j]+=np.random.uniform(-1,1)*0.01*noisy[i,j]
  print(np.amax(noisy[:,1:]))
  noisy[:,1:]=noisy[:,1:]/np.amax(noisy[:,1:])*smax
  noisy[:,1:][noisy[:,1:] < 0] = 0
  noisy[:,1:]=np.rint(noisy[:,1:])
  #print(np.shape(noisy))
  return noisy


In [212]:
files=glob.glob("csv/png/sel_obs2nm_blank_3000ul*.csv")

In [213]:
    fileslist=dict.fromkeys(files,1)
    i=1
    for key in fileslist:
        td=pd.read_csv(key,index_col=0)
        # col[key[4:-4]+'.txt']=td.columns
        tdn=np.array(td)
        datadict[key[4:-4]+'.txt']=tdn.astype(np.float)
        i+=1

In [214]:
def add_blank(x):
  if 'blank' not in x.file_name:
    ret=dfblank.loc[(dfblank['typ'] == x['typ']) & (dfblank['alcohol_concentration'] == x['alcohol_concentration']),'spec'].iloc[0]
    print(ret)
  else:
    ret=np.nan
  return np.rint(ret)

In [215]:
def fill(x,source):
  y=x
  for col in source.columns:

    if np.isnan(np.amax(np.amax(x[col]))):
      nancheck=0
      counter=0
      while nancheck==0:
        newval=source[col].iloc[np.random.randint(0,len(source))]
        counter+=1
        if not np.isnan(np.amax(np.amax(newval))):
          print('yes')
          nancheck=1
          newval=noise(newval)
        if counter==20:
          newval=np.nan
          break
     # print(np.shape(noise(newval)))
      y[col]=newval    
  return y

In [216]:
dfsamples=pd.read_pickle('csv/PL_LT_beverage_pd.pkl')

In [217]:
mapmax,diffmax,LT405max,LT450max=maxima(datadict)
dnormdict=norm_int(datadict,mapmax)
checker4='sel_obs2nm_blank_3000ul(.*)_(.*)_(.*).txt'
dfblank=pd.DataFrame(index=None,columns=('alcohol_concentration','CD_amount','typ','file_name','spec'))
for key in dnormdict: 
  if re.search(checker4,key) is not None: 
    con=re.search(checker4,key).group(1)
    am='blank'
    typ=re.search(checker4,key).group(2)
    print(am)
    dat=pd.DataFrame([[con,am,typ,key,dnormdict[key]]]) 
    dat.columns=dfblank.columns 
    dfblank=dfblank.append(dat,ignore_index=True) 
dfblank=dfblank[dfblank.alcohol_concentration != 'ethanol_0ulwater']
dfblank=dfblank[dfblank.alcohol_concentration != 'eth+h2o']
	

65535.0
2999.0
65535.0
3575.0
11118.0
13178.0
1
13178.0
65535.0
65535.0
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank
blank


In [218]:
dfsamples=dfsamples.drop(dfsamples.loc[dfsamples['typ']=='LT'].index)

In [219]:
dfsamples=pd.concat([dfsamples,dfblank])

In [220]:
dfblank=dfblank.drop(dfblank.loc[dfblank['CD_amount']=='blank'].index)

In [221]:
dfsamples['LT405spec'] = dfsamples.apply(lambda x: x['spec'] if x['typ']=='LT405' else np.nan, axis=1)
dfsamples['LT450spec'] = dfsamples.apply(lambda x: x['spec'] if x['typ']=='LT450' else np.nan, axis=1)
dfsamples['PLspec'] = dfsamples.apply(lambda x: x['spec'] if x['typ']=='PL' else np.nan, axis=1)


In [222]:
for amount in dfsamples['CD_amount'].unique():
  print(amount)
  dfsamples[str(amount)+'_LT405spec'] = dfsamples.apply(lambda x: x['LT405spec'] if x['CD_amount']==(amount) else np.nan, axis=1)
  dfsamples[str(amount)+'_LT450spec'] = dfsamples.apply(lambda x: x['LT450spec'] if x['CD_amount']==(amount) else np.nan, axis=1)
  dfsamples[str(amount)+'_PLspec'] = dfsamples.apply(lambda x: x['PLspec'] if x['CD_amount']==(amount) else np.nan, axis=1)

100.0
150.0
300.0
blank


In [223]:
dfsamples=dfsamples.drop('LT405spec',axis=1)
dfsamples=dfsamples.drop('LT450spec',axis=1)
dfsamples=dfsamples.drop('PLspec',axis=1)

In [224]:
dfsamples=dfsamples.drop('spec',axis=1)

In [225]:
cl=dfsamples.iloc[:,4:].columns
c=dfsamples.iloc[:,7:-3].columns

In [226]:
df4=pd.DataFrame(columns=dfsamples.columns)

In [227]:

for alccon in dfsamples['alcohol_concentration'].unique():
  print(alccon)
  df_tmp=copy.deepcopy(dfsamples.loc[dfsamples['alcohol_concentration']==alccon])
  df_tmp= df_tmp.apply(lambda x: pd.Series(x.dropna().values))
  print('drop')
  no=max(df_tmp[cl].count())
  df_tmp=df_tmp.iloc[:no,:]
  df_tmp=df_tmp.apply(lambda x: fill(x,df_tmp[cl]) ,axis=1)
  df4=pd.concat((df4,df_tmp))
  del df_tmp
  #df3[str(int(amount))+'_LT405spec'] = df3.apply(lambda x: x['LT405spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  #df3[str(int(amount))+'_LT450spec'] = df3.apply(lambda x: x['LT450spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  #df3[str(int(amount))+'_PLspec'] = df3.apply(lambda x: x['PLspec'] if x['CD_amount']==int(amount) else np.nan, axis=1)

limoncello
drop
luzhoulaojiao
drop
yes
65107
yes
65158
yes
63623
yes
65068
yes
65532
yes
65338
yes
65032
yes
65535
yes
65404
yes
65015
yes
65530
yes
64944
yes
65516
yes
65040
yes
64584
yes
65531
bitburgerdrive
drop
yes
65486
yes
65387
yes
65451
yes
65534
yes
64772
yes
65111
yes
65535
yes
65350
yes
65012
yes
65533
yes
64937
yes
65527
yes
64692
yes
65424
yes
65535
yes
64421
yes
65160
yes
65530
chang
drop
absolut
drop
yes
65086
yes
65319
yes
65533
yes
65481
yes
64354
yes
65532
yes
65220
yes
65533
yes
64172
yes
63980
yes
65535
whitewine
drop
yes
65441
yes
63862
yes
65531
yes
65013
yes
65132
yes
65047
yes
64136
yes
65535
yes
64936
yes
64889
yes
65535
bitburgerpils
drop
yes
63400
yes
65218
yes
65530
yes
65481
yes
65314
yes
65526
yes
64346
yes
62375
yes
65141
yes
64970
yes
65532
yes
63973
yes
65004
yes
65534
yes
62760
yes
64239
yes
65534
chimay
drop
bombaygin
drop
whitewine2
drop
yes
64888
yes
65069
yes
65533
yes
65161
yes
65535
yes
64563
yes
64432
yes
65535
yes
65304
yes
65405
yes
65531


In [228]:
df4=df4.reset_index(drop=True)
df4=df4.drop(['CD_amount','typ','file_name'],axis=1)

In [229]:
df4.to_pickle('PL_LT_beverage_onemat_pd_filledblank.pkl')

In [230]:
#df4=pd.read_pickle('PL_LT_beverage_onemat_pd.pkl')

In [231]:
df4

,alcohol_concentration,100.0_LT405spec,100.0_LT450spec,100.0_PLspec,150.0_LT405spec,150.0_LT450spec,150.0_PLspec,300.0_LT405spec,300.0_LT450spec,300.0_PLspec,blank_LT405spec,blank_LT450spec,blank_PLspec
0,limoncello,"[[0, 57], [1, 57], [2, 66], [3, 62], [4, 64], ...","[[0, 56], [1, 56], [2, 59], [3, 68], [4, 47], ...","[[300, 0, 7, 10, 0, 1, 14, 2, 0, 12, 0, 0, 7, ...","[[0, 61], [1, 61], [2, 70], [3, 72], [4, 68], ...","[[0, 49], [1, 49], [2, 62], [3, 62], [4, 55], ...","[[300, 0, 6, 2, 0, 0, 0, 15, 0, 0, 0, 10, 7, 0...","[[0, 57], [1, 57], [2, 68], [3, 55], [4, 55], ...","[[0, 52], [1, 52], [2, 60], [3, 66], [4, 75], ...","[[300, 2, 0, 3, 5, 0, 14, 0, 0, 0, 0, 13, 0, 1...","[[0, 42], [1, 42], [2, 45], [3, 37], [4, 39], ...","[[0, 37], [1, 37], [2, 29], [3, 41], [4, 36], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 5, 0, 20, 0, 0, 10..."
1,luzhoulaojiao,"[[0, 66], [1, 66], [2, 66], [3, 73], [4, 65], ...","[[0, 88], [1, 88], [2, 83], [3, 98], [4, 100],...","[[300, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 7, ...","[[0, 91], [1, 91], [2, 93], [3, 91], [4, 87], ...","[[0, 100], [1, 100], [2, 100], [3, 96], [4, 11...","[[300, 0, 0, 0, 9, 0, 1, 7, 11, 20, 11, 23, 29...","[[0, 278], [1, 278], [2, 240], [3, 272], [4, 2...","[[0, 130], [1, 130], [2, 167], [3, 148], [4, 1...","[[300, 0, 0, 8, 14, 0, 0, 1, 0, 0, 14, 0, 0, 1...","[[0, 97], [1, 97], [2, 102], [3, 98], [4, 97],...","[[0, 320], [1, 320], [2, 379], [3, 424], [4, 4...","[[300, 786, 816, 776, 1084, 1930, 2352, 2014, ..."
2,luzhoulaojiao,"[[0, 89], [1, 89], [2, 87], [3, 98], [4, 102],...","[[0, 105], [1, 105], [2, 96], [3, 98], [4, 95]...","[[300, 9, 37, 101, 123, 135, 142, 139, 160, 19...","[[0, 91], [1, 88], [2, 92], [3, 91], [4, 87], ...","[[0, 99], [1, 100], [2, 98], [3, 93], [4, 114]...","[[300, 2, 8, 7, 15, 4, 11, 0, 17, 13, 13, 2, 2...","[[0, 156], [1, 156], [2, 163], [3, 137], [4, 1...","[[0, 194], [1, 194], [2, 232], [3, 247], [4, 2...","[[300, 8, 0, 6, 0, 2, 2, 0, 0, 0, 0, 3, 5, 0, ...","[[0, 97], [1, 96], [2, 101], [3, 99], [4, 99],...","[[0, 321], [1, 323], [2, 379], [3, 426], [4, 4...","[[300, 42, 43, 42, 58, 104, 127, 109, 111, 90,..."
3,luzhoulaojiao,"[[0, 89], [1, 86], [2, 85], [3, 95], [4, 102],...","[[0, 86], [1, 89], [2, 82], [3, 97], [4, 99], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 55, 0, 15031, 126,...","[[0, 91], [1, 89], [2, 92], [3, 91], [4, 87], ...","[[0, 101], [1, 98], [2, 100], [3, 96], [4, 111...","[[300, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, ...","[[0, 281], [1, 280], [2, 242], [3, 276], [4, 2...","[[0, 192], [1, 192], [2, 228], [3, 246], [4, 2...","[[300, 0, 11, 0, 0, 0, 3, 9, 0, 0, 0, 2, 0, 0,...","[[0, 96], [1, 97], [2, 102], [3, 96], [4, 98],...","[[0, 322], [1, 327], [2, 387], [3, 433], [4, 4...","[[300, 2, 2, 2, 3, 5, 6, 5, 5, 4, 4, 2, 3, 2, ..."
4,bitburgerdrive,"[[0, 73], [1, 73], [2, 79], [3, 101], [4, 90],...","[[0, 102], [1, 102], [2, 109], [3, 88], [4, 10...","[[300, 0, 5, 8, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...","[[0, 136], [1, 136], [2, 134], [3, 128], [4, 1...","[[0, 75], [1, 75], [2, 73], [3, 69], [4, 75], ...","[[300, 6, 0, 2, 2, 8, 0, 2, 0, 0, 2, 3, 0, 9, ...","[[0, 75], [1, 75], [2, 83], [3, 97], [4, 77], ...","[[0, 68], [1, 68], [2, 106], [3, 96], [4, 98],...","[[300, 0, 0, 4, 0, 0, 2, 5, 0, 0, 0, 0, 11, 0,...","[[0, 87], [1, 87], [2, 83], [3, 85], [4, 86], ...","[[0, 73], [1, 73], [2, 87], [3, 75], [4, 87], ...","[[300, 0, 0, 15, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0,..."
5,bitburgerdrive,"[[0, 68], [1, 68], [2, 80], [3, 76], [4, 75], ...","[[0, 61], [1, 61], [2, 77], [3, 75], [4, 73], ...","[[300, 0, 0, 0, 0, 6, 8, 3, 2, 0, 0, 5, 0, 0, ...","[[0, 97], [1, 97], [2, 92], [3, 96], [4, 95], ...","[[0, 74], [1, 73], [2, 73], [3, 67], [4, 73], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 75], [1, 73], [2, 81], [3, 95], [4, 78], ...","[[0, 68], [1, 69], [2, 103], [3, 95], [4, 96],...","[[300, 898, 901, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 86], [1, 86], [2, 82], [3, 84], [4, 84], ...","[[0, 72], [1, 70], [2, 86], [3, 74], [4, 85], ...","[[300, 57

In [232]:
emptyLT=np.concatenate((np.linspace(0,1023,1024)[:,np.newaxis],np.linspace(0,0,1024)[:,np.newaxis]),axis=1).astype(np.uint16)
emptyPL=np.concatenate((np.linspace(300,600,16)[:,np.newaxis],np.zeros((16,216))),axis=1).astype(np.uint16)

In [233]:
#df4.at[4,'blank_PLspec']=emptyPL
df4['alcohol_concentration_2']=np.nan


In [234]:
df4['alcohol_concentration_2']=df4.apply(lambda x: float(4.8) if 'bitburgerpils' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(0.0) if 'bitburgerdrive' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(47.0) if 'bombaygin' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(52.0) if 'luzhoulaojiao' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(13.5) if 'whitewine' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(13.0) if 'whitewine2' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(5.0) if 'chang' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(40.0) if 'absolut' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(9.0) if 'chimay' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)
df4['alcohol_concentration_2']=df4.apply(lambda x: float(30.0) if 'limoncello' in str(x['alcohol_concentration']) else x['alcohol_concentration_2'],axis=1)

In [235]:
df4.rename(columns={'300.0_LT405spec':'300_LT405spec',
 '300.0_LT450spec':'300_LT450spec',
 '300.0_PLspec':'300_PLspec',
 '150.0_LT405spec':'150_LT405spec',
 '150.0_LT450spec':'150_LT450spec',
 '150.0_PLspec':'150_PLspec',
 '100.0_LT405spec':'100_LT405spec',
 '100.0_LT450spec':'100_LT450spec',
 '100.0_PLspec':'100_PLspec'}, inplace=True)

In [236]:
df4

,alcohol_concentration,100_LT405spec,100_LT450spec,100_PLspec,150_LT405spec,150_LT450spec,150_PLspec,300_LT405spec,300_LT450spec,300_PLspec,blank_LT405spec,blank_LT450spec,blank_PLspec,alcohol_concentration_2
0,limoncello,"[[0, 57], [1, 57], [2, 66], [3, 62], [4, 64], ...","[[0, 56], [1, 56], [2, 59], [3, 68], [4, 47], ...","[[300, 0, 7, 10, 0, 1, 14, 2, 0, 12, 0, 0, 7, ...","[[0, 61], [1, 61], [2, 70], [3, 72], [4, 68], ...","[[0, 49], [1, 49], [2, 62], [3, 62], [4, 55], ...","[[300, 0, 6, 2, 0, 0, 0, 15, 0, 0, 0, 10, 7, 0...","[[0, 57], [1, 57], [2, 68], [3, 55], [4, 55], ...","[[0, 52], [1, 52], [2, 60], [3, 66], [4, 75], ...","[[300, 2, 0, 3, 5, 0, 14, 0, 0, 0, 0, 13, 0, 1...","[[0, 42], [1, 42], [2, 45], [3, 37], [4, 39], ...","[[0, 37], [1, 37], [2, 29], [3, 41], [4, 36], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 5, 0, 20, 0, 0, 10...",30.0
1,luzhoulaojiao,"[[0, 66], [1, 66], [2, 66], [3, 73], [4, 65], ...","[[0, 88], [1, 88], [2, 83], [3, 98], [4, 100],...","[[300, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 7, ...","[[0, 91], [1, 91], [2, 93], [3, 91], [4, 87], ...","[[0, 100], [1, 100], [2, 100], [3, 96], [4, 11...","[[300, 0, 0, 0, 9, 0, 1, 7, 11, 20, 11, 23, 29...","[[0, 278], [1, 278], [2, 240], [3, 272], [4, 2...","[[0, 130], [1, 130], [2, 167], [3, 148], [4, 1...","[[300, 0, 0, 8, 14, 0, 0, 1, 0, 0, 14, 0, 0, 1...","[[0, 97], [1, 97], [2, 102], [3, 98], [4, 97],...","[[0, 320], [1, 320], [2, 379], [3, 424], [4, 4...","[[300, 786, 816, 776, 1084, 1930, 2352, 2014, ...",52.0
2,luzhoulaojiao,"[[0, 89], [1, 89], [2, 87], [3, 98], [4, 102],...","[[0, 105], [1, 105], [2, 96], [3, 98], [4, 95]...","[[300, 9, 37, 101, 123, 135, 142, 139, 160, 19...","[[0, 91], [1, 88], [2, 92], [3, 91], [4, 87], ...","[[0, 99], [1, 100], [2, 98], [3, 93], [4, 114]...","[[300, 2, 8, 7, 15, 4, 11, 0, 17, 13, 13, 2, 2...","[[0, 156], [1, 156], [2, 163], [3, 137], [4, 1...","[[0, 194], [1, 194], [2, 232], [3, 247], [4, 2...","[[300, 8, 0, 6, 0, 2, 2, 0, 0, 0, 0, 3, 5, 0, ...","[[0, 97], [1, 96], [2, 101], [3, 99], [4, 99],...","[[0, 321], [1, 323], [2, 379], [3, 426], [4, 4...","[[300, 42, 43, 42, 58, 104, 127, 109, 111, 90,...",52.0
3,luzhoulaojiao,"[[0, 89], [1, 86], [2, 85], [3, 95], [4, 102],...","[[0, 86], [1, 89], [2, 82], [3, 97], [4, 99], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 55, 0, 15031, 126,...","[[0, 91], [1, 89], [2, 92], [3, 91], [4, 87], ...","[[0, 101], [1, 98], [2, 100], [3, 96], [4, 111...","[[300, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, ...","[[0, 281], [1, 280], [2, 242], [3, 276], [4, 2...","[[0, 192], [1, 192], [2, 228], [3, 246], [4, 2...","[[300, 0, 11, 0, 0, 0, 3, 9, 0, 0, 0, 2, 0, 0,...","[[0, 96], [1, 97], [2, 102], [3, 96], [4, 98],...","[[0, 322], [1, 327], [2, 387], [3, 433], [4, 4...","[[300, 2, 2, 2, 3, 5, 6, 5, 5, 4, 4, 2, 3, 2, ...",52.0
4,bitburgerdrive,"[[0, 73], [1, 73], [2, 79], [3, 101], [4, 90],...","[[0, 102], [1, 102], [2, 109], [3, 88], [4, 10...","[[300, 0, 5, 8, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...","[[0, 136], [1, 136], [2, 134], [3, 128], [4, 1...","[[0, 75], [1, 75], [2, 73], [3, 69], [4, 75], ...","[[300, 6, 0, 2, 2, 8, 0, 2, 0, 0, 2, 3, 0, 9, ...","[[0, 75], [1, 75], [2, 83], [3, 97], [4, 77], ...","[[0, 68], [1, 68], [2, 106], [3, 96], [4, 98],...","[[300, 0, 0, 4, 0, 0, 2, 5, 0, 0, 0, 0, 11, 0,...","[[0, 87], [1, 87], [2, 83], [3, 85], [4, 86], ...","[[0, 73], [1, 73], [2, 87], [3, 75], [4, 87], ...","[[300, 0, 0, 15, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0,...",0.0
5,bitburgerdrive,"[[0, 68], [1, 68], [2, 80], [3, 76], [4, 75], ...","[[0, 61], [1, 61], [2, 77], [3, 75], [4, 73], ...","[[300, 0, 0, 0, 0, 6, 8, 3, 2, 0, 0, 5, 0, 0, ...","[[0, 97], [1, 97], [2, 92], [3, 96], [4, 95], ...","[[0, 74], [1, 73], [2, 73], [3, 67], [4, 73], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 75], [1, 73], [2, 81], [3, 95], [4, 78], ...","[[0, 68], [1, 69], [2, 103], [3, 95], [4, 96],...","[[300, 898, 901, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 86], [1, 86], [2, 82], [3, 84], [4, 84], ...","[[0, 72], [1, 70], [2, 86], [3,

In [237]:
list(df4.columns.values)
df4 = df4[['alcohol_concentration',
'alcohol_concentration_2',
 'blank_LT405spec',
 'blank_LT450spec',
 'blank_PLspec',
 '300_LT405spec',
 '300_LT450spec',
 '300_PLspec',
 '150_LT405spec',
 '150_LT450spec',
 '150_PLspec',
 '100_LT405spec',
 '100_LT450spec',
 '100_PLspec'
 ]]

In [238]:
df4.rename(columns={'alcohol_concentration':'beverage'}, inplace=True)
df4.rename(columns={'alcohol_concentration_2':'alcohol_concentration'}, inplace=True)

In [240]:
df4.to_pickle('PL_LT_beverage_onemat_pd_2.pkl')

In [239]:
df4

,beverage,alcohol_concentration,blank_LT405spec,blank_LT450spec,blank_PLspec,300_LT405spec,300_LT450spec,300_PLspec,150_LT405spec,150_LT450spec,150_PLspec,100_LT405spec,100_LT450spec,100_PLspec
0,limoncello,30.0,"[[0, 42], [1, 42], [2, 45], [3, 37], [4, 39], ...","[[0, 37], [1, 37], [2, 29], [3, 41], [4, 36], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 5, 0, 20, 0, 0, 10...","[[0, 57], [1, 57], [2, 68], [3, 55], [4, 55], ...","[[0, 52], [1, 52], [2, 60], [3, 66], [4, 75], ...","[[300, 2, 0, 3, 5, 0, 14, 0, 0, 0, 0, 13, 0, 1...","[[0, 61], [1, 61], [2, 70], [3, 72], [4, 68], ...","[[0, 49], [1, 49], [2, 62], [3, 62], [4, 55], ...","[[300, 0, 6, 2, 0, 0, 0, 15, 0, 0, 0, 10, 7, 0...","[[0, 57], [1, 57], [2, 66], [3, 62], [4, 64], ...","[[0, 56], [1, 56], [2, 59], [3, 68], [4, 47], ...","[[300, 0, 7, 10, 0, 1, 14, 2, 0, 12, 0, 0, 7, ..."
1,luzhoulaojiao,52.0,"[[0, 97], [1, 97], [2, 102], [3, 98], [4, 97],...","[[0, 320], [1, 320], [2, 379], [3, 424], [4, 4...","[[300, 786, 816, 776, 1084, 1930, 2352, 2014, ...","[[0, 278], [1, 278], [2, 240], [3, 272], [4, 2...","[[0, 130], [1, 130], [2, 167], [3, 148], [4, 1...","[[300, 0, 0, 8, 14, 0, 0, 1, 0, 0, 14, 0, 0, 1...","[[0, 91], [1, 91], [2, 93], [3, 91], [4, 87], ...","[[0, 100], [1, 100], [2, 100], [3, 96], [4, 11...","[[300, 0, 0, 0, 9, 0, 1, 7, 11, 20, 11, 23, 29...","[[0, 66], [1, 66], [2, 66], [3, 73], [4, 65], ...","[[0, 88], [1, 88], [2, 83], [3, 98], [4, 100],...","[[300, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 7, ..."
2,luzhoulaojiao,52.0,"[[0, 97], [1, 96], [2, 101], [3, 99], [4, 99],...","[[0, 321], [1, 323], [2, 379], [3, 426], [4, 4...","[[300, 42, 43, 42, 58, 104, 127, 109, 111, 90,...","[[0, 156], [1, 156], [2, 163], [3, 137], [4, 1...","[[0, 194], [1, 194], [2, 232], [3, 247], [4, 2...","[[300, 8, 0, 6, 0, 2, 2, 0, 0, 0, 0, 3, 5, 0, ...","[[0, 91], [1, 88], [2, 92], [3, 91], [4, 87], ...","[[0, 99], [1, 100], [2, 98], [3, 93], [4, 114]...","[[300, 2, 8, 7, 15, 4, 11, 0, 17, 13, 13, 2, 2...","[[0, 89], [1, 89], [2, 87], [3, 98], [4, 102],...","[[0, 105], [1, 105], [2, 96], [3, 98], [4, 95]...","[[300, 9, 37, 101, 123, 135, 142, 139, 160, 19..."
3,luzhoulaojiao,52.0,"[[0, 96], [1, 97], [2, 102], [3, 96], [4, 98],...","[[0, 322], [1, 327], [2, 387], [3, 433], [4, 4...","[[300, 2, 2, 2, 3, 5, 6, 5, 5, 4, 4, 2, 3, 2, ...","[[0, 281], [1, 280], [2, 242], [3, 276], [4, 2...","[[0, 192], [1, 192], [2, 228], [3, 246], [4, 2...","[[300, 0, 11, 0, 0, 0, 3, 9, 0, 0, 0, 2, 0, 0,...","[[0, 91], [1, 89], [2, 92], [3, 91], [4, 87], ...","[[0, 101], [1, 98], [2, 100], [3, 96], [4, 111...","[[300, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, ...","[[0, 89], [1, 86], [2, 85], [3, 95], [4, 102],...","[[0, 86], [1, 89], [2, 82], [3, 97], [4, 99], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 55, 0, 15031, 126,..."
4,bitburgerdrive,0.0,"[[0, 87], [1, 87], [2, 83], [3, 85], [4, 86], ...","[[0, 73], [1, 73], [2, 87], [3, 75], [4, 87], ...","[[300, 0, 0, 15, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0,...","[[0, 75], [1, 75], [2, 83], [3, 97], [4, 77], ...","[[0, 68], [1, 68], [2, 106], [3, 96], [4, 98],...","[[300, 0, 0, 4, 0, 0, 2, 5, 0, 0, 0, 0, 11, 0,...","[[0, 136], [1, 136], [2, 134], [3, 128], [4, 1...","[[0, 75], [1, 75], [2, 73], [3, 69], [4, 75], ...","[[300, 6, 0, 2, 2, 8, 0, 2, 0, 0, 2, 3, 0, 9, ...","[[0, 73], [1, 73], [2, 79], [3, 101], [4, 90],...","[[0, 102], [1, 102], [2, 109], [3, 88], [4, 10...","[[300, 0, 5, 8, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ..."
5,bitburgerdrive,0.0,"[[0, 86], [1, 86], [2, 82], [3, 84], [4, 84], ...","[[0, 72], [1, 70], [2, 86], [3, 74], [4, 85], ...","[[300, 57404, 0, 10, 39, 158, 57385, 3, 4, 0, ...","[[0, 75], [1, 73], [2, 81], [3, 95], [4, 78], ...","[[0, 68], [1, 69], [2, 103], [3, 95], [4, 96],...","[[300, 898, 901, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 97], [1, 97], [2, 92], [3, 96], [4, 95], ...","[[0, 74], [1, 73], [2, 73], [3, 67], [4, 73], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 68], [1, 68], [2, 80], [3, 76], [4, 75], ...","[[0, 61], [1, 61], [2, 77], [3, 75], [4, 7